In [1]:
PATH_TO_INITIAL_STATE = '/Users/pmikheev/land_storage_assignment/data/storage_asof_20200101.parquet'

In [4]:
import pandas as pd

In [5]:
!pip install pyarrow
!pip install fastparquet

  Using cached pyarrow-7.0.0-cp39-cp39-macosx_10_13_x86_64.whl (20.2 MB)
  Using cached fastparquet-0.8.1-cp39-cp39-macosx_10_9_universal2.whl (775 kB)
  Using cached cramjam-2.5.0-cp39-cp39-macosx_10_9_x86_64.macosx_11_0_arm64.macosx_10_9_universal2.whl (2.7 MB)


In [71]:
init_df = pd.read_parquet(PATH_TO_INITIAL_STATE)

In [72]:
init_df.sort_values(['crude_relative', 'diesel_relative'])

,port,crude_absolute,crude_relative,diesel_absolute,diesel_relative
2,Rotterdam,42000000,0.33,72000000,0.81
0,Houston,50000000,0.45,2250000,0.33
1,Singapore,120000000,0.50,15000000,0.62


In [14]:
init_df.shape[0]

3

In [5]:
PATH_TO_EVENT_STATE = '/Users/pmikheev/land_storage_assignment/data/cargo_movements.parquet'

event_df = pd.read_parquet(PATH_TO_EVENT_STATE)

In [6]:
event_df.shape[0]

100

In [7]:
event_df.sort_values(['start_timestamp', 'end_timestamp'])

,loading_port,start_timestamp,discharge_port,end_timestamp,product,quantity
4,Houston,2020-01-01 05:23:31,Yeosu,2020-01-05 13:35:20,fuel oil,122089
88,Houston,2020-01-01 05:28:02,Lagos,2020-01-12 14:08:10,fuel oil,55940
89,Rotterdam,2020-01-01 07:53:09,Yeosu,2020-01-10 07:32:44,diesel,21914
81,Rotterdam,2020-01-01 08:54:06,Yeosu,2020-01-12 14:41:25,fuel oil,35951
82,Lagos,2020-01-01 17:06:43,Singapore,2020-01-05 22:34:28,crude oil,198300
...,...,...,...,...,...,...
26,Lagos,2020-01-14 05:12:11,Yeosu,2020-01-26 20:32:39,crude oil,17921
95,Singapore,2020-01-14 07:30:15,Lagos,2020-01-20 01:02:43,fuel oil,191392
96,Yeosu,2020-01-14 09:17:16,Yeosu,2020-01-22 18:01:22,crude oil,36744
59,Rotterdam,2020-01-14 14:50:56,Rotterdam,2020-01-27 23:51:32,crude oil,178572


In [8]:
loading_df = event_df.loc[:, ['loading_port', 'start_timestamp', 'product', 'quantity']].rename(columns={
    'start_timestamp': 'ts', 
    'loading_port': 'port', 
})
loading_df['action'] = 'load'

In [9]:
discharge_df = event_df.loc[:, ['discharge_port', 'end_timestamp', 'product', 'quantity']].rename(columns={'end_timestamp': 'ts', 'discharge_port': 'port'})
discharge_df['action'] = 'discharge'

In [10]:
loading_df.head(5)

,port,ts,product,quantity,action
0,Rotterdam,2020-01-12 00:00:59,diesel,16065,load
1,Lagos,2020-01-03 15:13:47,fuel oil,188414,load
2,Houston,2020-01-08 19:50:22,crude oil,165326,load
3,Rotterdam,2020-01-05 22:22:21,diesel,42576,load
4,Houston,2020-01-01 05:23:31,fuel oil,122089,load


In [11]:
discharge_df.head(5)

,port,ts,product,quantity,action
0,Singapore,2020-01-24 09:29:29,diesel,16065,discharge
1,Yeosu,2020-01-12 21:45:42,fuel oil,188414,discharge
2,Lagos,2020-01-22 01:17:00,crude oil,165326,discharge
3,Houston,2020-01-14 09:27:04,diesel,42576,discharge
4,Yeosu,2020-01-05 13:35:20,fuel oil,122089,discharge


In [12]:
merged_df = pd.concat([loading_df, discharge_df]).sort_values('ts')
merged_df

,port,ts,product,quantity,action
4,Houston,2020-01-01 05:23:31,fuel oil,122089,load
88,Houston,2020-01-01 05:28:02,fuel oil,55940,load
89,Rotterdam,2020-01-01 07:53:09,diesel,21914,load
81,Rotterdam,2020-01-01 08:54:06,fuel oil,35951,load
82,Lagos,2020-01-01 17:06:43,crude oil,198300,load
...,...,...,...,...,...
37,Rotterdam,2020-01-24 11:13:30,crude oil,164829,discharge
79,Singapore,2020-01-25 22:17:51,diesel,181982,discharge
19,Rotterdam,2020-01-26 02:29:04,fuel oil,61704,discharge
26,Yeosu,2020-01-26 20:32:39,crude oil,17921,discharge


In [38]:
# merged_df[merged_df['product'] == 'crude oil' and merged_df['port'] in ['Rotterdam', 'Houston', 'Singapore']].shape[0]
merged_df[((merged_df['product'] == 'crude oil') | (merged_df['product'] == 'diesel')) & ((merged_df['port'] == 'Rotterdam') | (merged_df['port'] == 'Houston') | (merged_df['port'] == 'Singapore'))].shape[0]



69

In [28]:
merged_df[merged_df['product'] == 'diesel'].shape[0]

70

In [26]:
merged_df.shape[0]

200

In [13]:
from dataclasses import dataclass
from datetime import datetime

from enum import Enum


class ProductType(Enum):
    CRUDE_OIL = 'crude oil'
    DIESEL = 'diesel'


class ActionType(Enum):
    LOAD = 'load'
    DISCHARGE = 'discharge'


@dataclass
class Event:
    action: ActionType
    product: ProductType
    port: str
    quantity: int
    ts: datetime


In [14]:
for idx, row in merged_df.iterrows():
    print(row['action'])
    r = Event(**row.to_dict())
    print(r)
    print(type(r.product))
    

load
Event(action='load', product='fuel oil', port='Houston', quantity=122089, ts=Timestamp('2020-01-01 05:23:31'))
<class 'str'>
load
Event(action='load', product='fuel oil', port='Houston', quantity=55940, ts=Timestamp('2020-01-01 05:28:02'))
<class 'str'>
load
Event(action='load', product='diesel', port='Rotterdam', quantity=21914, ts=Timestamp('2020-01-01 07:53:09'))
<class 'str'>
load
Event(action='load', product='fuel oil', port='Rotterdam', quantity=35951, ts=Timestamp('2020-01-01 08:54:06'))
<class 'str'>
load
Event(action='load', product='crude oil', port='Lagos', quantity=198300, ts=Timestamp('2020-01-01 17:06:43'))
<class 'str'>
load
Event(action='load', product='fuel oil', port='Singapore', quantity=28745, ts=Timestamp('2020-01-01 17:24:20'))
<class 'str'>
load
Event(action='load', product='diesel', port='Lagos', quantity=128771, ts=Timestamp('2020-01-01 19:32:48'))
<class 'str'>
load
Event(action='load', product='crude oil', port='Houston', quantity=67236, ts=Timestamp('20